# All Asset Classifications

**Category:** Assets
**Scope:** Asset Classification alignment to component map
**Author:** Michael Moyer   
**Created:** 01-13-2026   
**Last Updated:** 01-13-2026  

**Purpose:** Align classification to component map

Notebook gathers the data, organizes, cleans and provides a brief report on it. 
        
**Use Case:** Use this notebook to assign acm component level mapping to asset classificaiton

---

In [1]:
NOTEBOOK_NAME = 'site_assets_all-asset-classes'
sql_file = f'{NOTEBOOK_NAME}.sql'

## Step 0: Load Packages and SQL Script

### 0.0 Package Load

In [2]:
import pandas as pd
import os
import pyodbc
import re

from dotenv import load_dotenv, find_dotenv
from pathlib import Path


import warnings
import time       # for timing longer cells (running the sql query)
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
# Automatically search for .env in parent directories
load_dotenv(find_dotenv())

# Load from environment variables
DSN = os.getenv('MAXIMO_DSN')
USER = os.getenv('MAXIMO_USER')
PASSWORD = os.getenv('MAXIMO_PASS')

print(f"DSN: {DSN}")
print(f"User: {USER}")

DSN: MX76PROD
User: maximo7


In [4]:
# Suppress the pandas warning
warnings.filterwarnings('ignore', category=UserWarning)

def run_query_from_file(sql_path: str) -> pd.DataFrame:
    conn = pyodbc.connect(f"DSN={DSN};UID={USER};PWD={PASSWORD}")
    query = open(sql_path).read()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [5]:
print(f"Notebook: {NOTEBOOK_NAME}.ipynb")
print(f"Matching Script: {sql_file}")

Notebook: site_assets_all-asset-classes.ipynb
Matching Script: site_assets_all-asset-classes.sql


In [6]:
# Start timing
start_time = time.time()

# Run the sql script
df_raw = run_query_from_file(sql_file)

# Calculate and print execution time
execution_time = time.time() - start_time
print(f"Query execution time: {execution_time:.2f} seconds \n")

Query execution time: 7.98 seconds 



In [7]:
print(f"Raw Data Frame Shape: {df_raw.shape[0]:,} rows, {df_raw.shape[1]} columns \n")
print("--- DataFrame Info---\n")
print(df_raw.info(), "\n\n--- null value detection---\n\n", df_raw.isna().sum())
#Memory Usage
print("")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Raw Data Frame Shape: 25,661 rows, 4 columns 

--- DataFrame Info---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25661 entries, 0 to 25660
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ASSETNUM     25661 non-null  object
 1   DESCRIPTION  25661 non-null  object
 2   ASSET_CLASS  25661 non-null  object
 3   DEPT         25661 non-null  object
dtypes: object(4)
memory usage: 802.0+ KB
None 

--- null value detection---

 ASSETNUM       0
DESCRIPTION    0
ASSET_CLASS    0
DEPT           0
dtype: int64

Memory usage: 6.27 MB


In [8]:
# Cell 1: Explore the asset classes
print("Unique asset classes:")
print(df_raw['ASSET_CLASS'].value_counts())
print(f"\nTotal unique classes: {df_raw['ASSET_CLASS'].nunique()}")

Unique asset classes:
ASSET_CLASS
Carrier            3981
Panel              1918
Not In List        1823
Conveyor           1272
Manipulator        1069
                   ... 
Dock Locks            1
Seals                 1
Elevator              1
Fastening             1
Pallet Conveyor       1
Name: count, Length: 129, dtype: int64

Total unique classes: 129


In [9]:
# Cell: Create blank crosswalk template

# Get all unique asset classes
asset_classes = sorted(df_raw['ASSET_CLASS'].unique())

# Get all component types from your rules
df_rules = pd.read_pickle('component_map.pkl')
component_types = df_rules['Component Type'].tolist()

# Create empty dataframe
crosswalk = pd.DataFrame(
    index=asset_classes,
    columns=component_types
)

# Fill with empty strings (easier to mark with 'x')
crosswalk = crosswalk.fillna('')

# Add a count column to show how many assets in each class
asset_counts = df_raw['ASSET_CLASS'].value_counts()
crosswalk.insert(0, 'Asset_Count', crosswalk.index.map(asset_counts))

# Save to CSV for editing
crosswalk.to_csv('asset_component_crosswalk_BLANK.csv')
print(f"✓ Blank crosswalk saved with {len(asset_classes)} asset classes")
print(f"✓ and {len(component_types)} component types")
print("\nOpen asset_component_crosswalk_BLANK.csv in Excel and mark 'x' where applicable")

✓ Blank crosswalk saved with 129 asset classes
✓ and 23 component types

Open asset_component_crosswalk_BLANK.csv in Excel and mark 'x' where applicable


In [10]:
# Cell: Convert filled crosswalk to component mapping

# Load your filled crosswalk
crosswalk_filled = pd.read_csv('asset_component_crosswalk_FILLED.csv', index_col=0)

# Drop the Asset_Count column (we don't need it for mapping)
crosswalk_filled = crosswalk_filled.drop('Asset_Count', axis=1)

# Convert to mapping dictionary
asset_to_component_map = {}

for asset_class in crosswalk_filled.index:
    # Get all columns where there's an 'x' (case insensitive)
    components = []
    for component_type in crosswalk_filled.columns:
        cell_value = str(crosswalk_filled.loc[asset_class, component_type]).strip().lower()
        if cell_value in ['x', 'X']:
            components.append(component_type)
    
    # Add to mapping (only if there are components)
    if components:
        asset_to_component_map[asset_class] = components

print(f"Created mapping for {len(asset_to_component_map)} asset classes")
print("\nSample mappings:")
for i, (asset_class, components) in enumerate(list(asset_to_component_map.items())[:5]):
    print(f"\n{asset_class}:")
    for comp in components:
        print(f"  - {comp}")

# Cell: Apply mapping to assets
df_raw['COMPONENT_TYPES'] = df_raw['ASSET_CLASS'].apply(
    lambda x: asset_to_component_map.get(x, [])
)

# Check coverage
has_components = df_raw['COMPONENT_TYPES'].apply(lambda x: len(x) > 0)
print("\n" + "="*60)
print(f"Assets with components: {has_components.sum()} ({has_components.sum()/len(df_raw)*100:.1f}%)")
print(f"Assets without components: {(~has_components).sum()}")

# Cell: Save final asset-component mapping
df_raw.to_pickle('asset_components.pkl')
print("\n✓ Final asset-component mapping saved to asset_components.pkl")

Created mapping for 69 asset classes

Sample mappings:

Actuator:
  - Pneumatic Systems
  - Electrical Connections/Terminations
  - Compressed Air Distribution

Air Conditioner:
  - Rolling Element Bearings (Grease)
  - Belt Drives
  - Fans / Blowers
  - Transformers
  - Process Heat Exchangers

Automated Lifts:
  - Electric Motors
  - Rolling Element Bearings (Oil)
  - Rolling Element Bearings (Grease)
  - Gearboxes / Speed Reducers

Automation Equipment:
  - Electric Motors
  - Electrical Connections/Terminations

Boiler:
  - Electrical Connections/Terminations
  - Transformers
  - Steam Traps
  - Process Heat Exchangers

Assets with components: 14243 (55.5%)
Assets without components: 11418

✓ Final asset-component mapping saved to asset_components.pkl
